In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Two-gene data (`data`)

In [ ]:
exp93_path = base_path/'2024.04.14_exp93'/'export'

plates = pd.DataFrame({
    'data_path': [exp93_path/'plate1', exp93_path/'plate2', exp93_path/'plate3',],
    'yaml_path': [exp93_path/'exp93_wells.yaml']*3,
    'biorep': [1, 2, 3],
    'exp': ['exp93']*3,
})

cache_path = rd.rootdir/'output'/'fig_architecture'/'data.gzip'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A','SNAP-647-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(metadata_path)
data = data.merge(metadata, how='left', on='construct')
metadata_construct2 = pd.read_excel(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct2-metadata.xlsx')
data = data.merge(metadata_construct2, how='left', on='construct2')
data['condition'] = data['construct'] + '_' + data['construct2']

# Rename far-red channel
data.rename(columns={'SNAP-647-A': 'iRFP-A'}, inplace=True)
display(data)

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='GEEC555'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing iRFP gate
gate_iRFP = 2.5e2
gates['iRFP-A'] = [gate_iRFP]

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'iRFP-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
df = data[(data['expressing']) & (data['construct']!='UT')]

In [ ]:
# Bin data and calculate statistics
by = ['condition','construct','construct2','biorep','exp']
df_quantiles, stats, _, fits = base.calculate_bins_stats(df, by=by)

df_quantiles = df_quantiles.merge(metadata, how='left', on='construct')
df_quantiles.sort_values(['design','ts_kind','ts_num'], inplace=True)
df_quantiles = df_quantiles.merge(metadata_construct2, how='left', on='construct2')

stats = stats.merge(metadata, how='left', on='construct')
stats.sort_values(['design','ts_kind','ts_num'], inplace=True)
stats = stats.merge(metadata_construct2, how='left', on='construct2')

fits = fits.merge(metadata, how='left', on='construct')
fits.sort_values(['design','ts_kind','ts_num'], inplace=True)
fits = fits.merge(metadata_construct2, how='left', on='construct2')


In [ ]:
# output range of 5-95 percentile
p_5 = df.groupby(by)[['output']].apply(lambda x: np.percentile(x, 5)).rename('output_5th').reset_index()
p_95 = df.groupby(by)[['output']].apply(lambda x: np.percentile(x, 95)).rename('output_95th').reset_index()
stats = stats.merge(p_5, how='left')
stats = stats.merge(p_95, how='left')
stats['output_range'] = stats['output_95th'] - stats['output_5th']
stats['output_range_log'] = stats['output_95th'].apply(np.log10) - stats['output_5th'].apply(np.log10)

# fraction within 1 order of magnitude (10x) around median
def get_high_low(df):
    median = df['output'].median()
    return df.loc[(df['output']>(median * 10**(-0.5))) & (df['output']<(median * 10**0.5)), 'output'].count() / df['output'].count()

fraction = df.groupby(by)[df.columns].apply(get_high_low).rename('fraction_within_10x').reset_index()
stats = stats.merge(fraction, how='left')

In [ ]:
metadata_comb = data.drop_duplicates('condition')[['construct','construct2','condition']]
metadata_comb = metadata_comb.merge(metadata, how='left', on='construct')
metadata_comb = metadata_comb.merge(metadata_construct2, how='left', on='construct2')

# Create color palette
metadata_comb.loc[metadata_comb['gene']=='2T', 'color'] = base.colors['green']
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='EF1a'), 'color'] = base.colors['purple']
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='U6'), 'color'] = base.colors['blue']
metadata_comb.loc[(metadata_comb['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata_comb.loc[metadata_comb['group'].isin(['base','marker']), 'color'] = 'black'

# markers
metadata_comb['markers'] = 'X'
metadata_comb.loc[metadata_comb['gene']=='1T', 'markers'] = 'o'
metadata_comb.loc[metadata_comb['gene']=='2T', 'markers'] = 'D'
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='EF1a'), 'markers'] = '^'
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='U6'), 'markers'] = 'v'

metadata_comb['condition'] = metadata_comb['construct'] + '_' + metadata_comb['construct2']
gene_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}
metadata_comb['ts_label'] = metadata_comb['ts_kind'].replace(ts_label)

metadata_dict = metadata_comb.set_index('condition').to_dict('dict')
two_gene_palette = metadata_dict['color']
two_gene_markers = metadata_dict['markers']

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 
                     'ytick.major.size': 3, 'xtick.major.size': 3, 'ytick.minor.size': 2, 'ytick.major.pad': 2, 'xtick.major.pad': 2, 
                     'lines.linewidth': 1,
                     'axes.spines.right': False, 'axes.spines.top': False, 'axes.labelpad': 2})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,6))
fig_gridspec = matplotlib.gridspec.GridSpec(3, 6, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[2,2,1.5], width_ratios=[1]*6)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:2]),
    'B': fig.add_subfigure(fig_gridspec[0,2:]),
    'C': fig.add_subfigure(fig_gridspec[1,:2]),
    'D': fig.add_subfigure(fig_gridspec[1,2:4]),
    'E': fig.add_subfigure(fig_gridspec[1,4:]),
    'F': fig.add_subfigure(fig_gridspec[2,:2]),
    'G': fig.add_subfigure(fig_gridspec[2,2:]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

scatter_kwargs = dict(s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

output_path = rd.rootdir/'output'/'fig_architecture'
fig_path = output_path/'fig_architecture.pdf'
fig.savefig(rd.outfile(fig_path))

In [ ]:
# two-gene architectures with 3'UTR target sites
stats_subset = stats[((stats['gene']=='1T') & (stats['design']==1) & (stats['group']=='controller')) |
                ((stats['gene']=='2T') & (stats['group']=='dual') & (stats['ts_loc']=='3\'')) |
                ((stats['gene']=='2V') & (stats['group']=='ts3') & (stats['construct2_promoter']=='EF1a')) |
                ((stats['group']=='base') & (stats['construct2_promoter']!='U6'))].copy()
stats_subset.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)
fits_subset = fits[((fits['gene']=='1T') & (fits['design']==1) & (fits['group']=='controller')) |
                ((fits['gene']=='2T') & (fits['group']=='dual') & (fits['ts_loc']=='3\'')) |
                ((fits['gene']=='2V') & (fits['group']=='ts3') & (fits['construct2_promoter']=='EF1a')) |
                ((fits['group']=='base') & (fits['construct2_promoter']!='U6'))].copy()
fits_subset.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)
df_quantiles_subset = df_quantiles[(((df_quantiles['gene']=='1T') & (df_quantiles['design']==1) & (df_quantiles['group']=='controller')) |
                        ((df_quantiles['gene']=='2T') & (df_quantiles['group']=='dual') & (df_quantiles['ts_loc']=='3\'')) |
                        ((df_quantiles['gene']=='2V') & (df_quantiles['group']=='ts3') & (df_quantiles['construct2_promoter']=='EF1a')) |
                        ((df_quantiles['group']=='base') & (df_quantiles['construct2_promoter']!='U6')))].copy()
df_quantiles_subset.sort_values(['gene','construct2_promoter'], inplace=True)


# shift xticks to add more space between architecture groups
condition_loc = {k:v for k,v in zip(plot_df['condition'].unique(), xtick_locs)}
stats_subset['condition_loc'] = stats_subset['condition'].replace(condition_loc)

condition_loc = {k:v for k,v in zip(fits_subset['condition'].unique(), xtick_locs)}
fits_subset['condition_loc'] = fits_subset['condition'].replace(condition_loc)

metadata_comb['condition_loc'] = metadata_comb['condition'].map(condition_loc)
m = metadata_comb.dropna()
m['condition_loc'] = m['condition_loc'].astype(str)

buffer = 0.6
gene_order = ['1T', '2T', '2V']
xlim_adjusted = (-0.5, len(stats_subset['condition'].unique())-0.5+buffer*(len(gene_order)-1))
xtick_locs = [0,1,2, 3+buffer,4+buffer, 5+buffer*2,6+buffer*2,7+buffer*2,]
scatter_kwargs2 = dict(s=4, jitter=0.3, linewidth=0.5, edgecolor='white', native_scale=True)

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.4, top=0.4, right=0.1)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.3))

plot_df = stats_subset
plot_df2 = fits_subset

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='output_gmean', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], **scatter_kwargs2)
ax.set(title='Mean', xlim=xlim_adjusted, xlabel='', ylabel='', yscale='log', xticks=xtick_locs)
rd.plot.generate_xticklabels(m.drop_duplicates('condition_loc'), 'condition_loc', ['ts_label'], annotate=False, ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='output_std', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], **scatter_kwargs2)
ax.set(title='Standard deviation', xlim=xlim_adjusted, xlabel='', ylabel='', yscale='log', xticks=xtick_locs)
rd.plot.generate_xticklabels(m.drop_duplicates('condition_loc'), 'condition_loc', ['ts_label'], annotate=False, ax=ax)

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='slope', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], **scatter_kwargs2)

ax.set(title='Slope', xlim=xlim_adjusted, xlabel='', ylim=(0,1.2), ylabel='', xticks=xtick_locs,
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'])
rd.plot.generate_xticklabels(m.drop_duplicates('condition_loc'), 'condition_loc', ['ts_label'], annotate=False, ax=ax)

for i,ax in enumerate(axes):
    ax.axvspan(2.5+buffer/2, 4.5+buffer*1.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
    ax.annotate(architecture_order[0], (1,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
    ax.annotate(architecture_order[1], (3.5+buffer,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
    ax.annotate(architecture_order[2], (6+buffer*2,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_title(ax.get_title(), pad=10)
    sns.despine(ax=ax)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.7, bottom=0.45, top=0.52, right=0.2)
ax = subfig.subplots(1,1,)

xlim = (1e1,1e5)
biorep = 2
plot_df = df_quantiles_subset[df_quantiles_subset['biorep']==2]

sns.kdeplot(data=plot_df, x='output', hue='condition', palette=two_gene_palette, ax=ax,
            cumulative=True, common_norm=False, legend=False, log_scale=True)
sns.kdeplot(data=df_quantiles[df_quantiles['group']=='marker'], x='output', hue='condition', palette=two_gene_palette,
            cumulative=True, common_norm=False, legend=False, log_scale=True, ax=ax, ls=':', zorder=1)
ax.set(xlim=xlim, xticks=np.logspace(1,5,5), ylabel='fraction of cells\nbelow given output', title='Cumulative distribution')
ax.grid(zorder=-1, color=base.get_light_color(base.get_light_color(base.colors['gray'])), which='both')
sns.despine(ax=ax)
ax.minorticks_off()

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.27, bottom=0.45, top=0.52, right=0.15)
ax = subfig.subplots(1,1,)

plot_df = stats_subset

# width of output range 5-95%
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='output_range', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], **scatter_kwargs2)

ax.set(yscale='log', ylabel='', xlim=xlim_adjusted, xticks=xtick_locs, xlabel='',
      title='Output range of\n5-95th percentile')
rd.plot.generate_xticklabels(m.drop_duplicates('condition_loc'), 'condition_loc', ['ts_label'], annotate=False, ax=ax)
sns.despine(ax=ax)

ax.axvspan(2.5+buffer/2, 4.5+buffer*1.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
ax.annotate(architecture_order[0], (1,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
ax.annotate(architecture_order[1], (3.5+buffer,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
ax.annotate(architecture_order[2], (6+buffer*2,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title(ax.get_title(), pad=10)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.27, bottom=0.45, top=0.52, right=0.15)
ax = subfig.subplots(1,1,)

plot_df = stats_subset

# % within 10x of median
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition_loc', y='fraction_within_10x', hue='condition', palette=two_gene_palette,
                    legend=False, ax=ax, marker=two_gene_markers[construct], **scatter_kwargs2)

ax.set(ylim=(0,1), ylabel='', xlim=xlim_adjusted, xticks=xtick_locs, xlabel='',
      title='Fraction within\n1 order of magnitude')
rd.plot.generate_xticklabels(m.drop_duplicates('condition_loc'), 'condition_loc', ['ts_label'], annotate=False, ax=ax)
sns.despine(ax=ax)

ax.axvspan(2.5+buffer/2, 4.5+buffer*1.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
ax.annotate(architecture_order[0], (1,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
ax.annotate(architecture_order[1], (3.5+buffer,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
ax.annotate(architecture_order[2], (6+buffer*2,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=smaller_size)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_title(ax.get_title(), pad=10)

fig.savefig(rd.outfile(fig_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.25, bottom=0.5, top=0.35, right=0.1)
axes = subfig.subplots(1,2,)

for ax in axes:
    ax.set(xlim=(1e1,1e5), xscale='log', xlabel='output', title='MOI = ')
    sns.despine(ax=ax)
    ax.minorticks_off()
    ax.get_yaxis().set_visible(False)

fig.savefig(rd.outfile(fig_path))